In [26]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from tqdm import tqdm
import re

In [27]:
# Funkcja do pobierania wyników z Google
def google_search(query, num_results=100):
    url = f"https://www.google.com/search?q={query}&num={num_results}"
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    }
    response = requests.get(url, headers=headers)
    
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        return soup
    else:
        print("Nie udało się pobrać wyników wyszukiwania")
        return None

# Funkcja do scrapowania danych kontaktowych
def scrape_contacts(soup):
    contacts = []
    
    # Wyszukiwanie linków w wynikach Google
    for idx, result in enumerate(soup.find_all('div', class_='g')):
        print(idx)
        title = result.find('h3')
        if title:
            name = title.get_text()
        else:
            name = "Brak nazwy"
        
        link = result.find('a')['href']
        
        # Pobieranie strony wynikowej
        try:
            contact_page = requests.get(link, headers={'User-Agent': 'Mozilla/5.0'})
            if contact_page.status_code != 200:
                print(f"Nie udało się otworzyć strony: {link}")
                continue
            
            contact_soup = BeautifulSoup(contact_page.text, 'html.parser')
        except Exception as e:
            print(f"Błąd podczas otwierania strony: {link} - {e}")
            continue  # Pomijamy stronę, jeśli nie uda się jej otworzyć

        #print(contact_soup)
        # Wyciąganie potencjalnych danych kontaktowych
        email = None
        phone = None
        
        # Szukaj maila
        for mail in contact_soup.find_all('a', href=True):
            
            if "mailto:" in mail['href']:
                
                email = re.search(r'([A-Za-z0-9._%+-]+@+[A-Za-z0-9._%+-]*\.(com|org|pl|at))', mail['href'])
                if email == None:
                    email = re.search(r'([A-Za-z0-9._%+-]+@+[A-Za-z0-9._%+-]*\.(com|org|pl|at))', mail.text)
                email = email.group(1)
                break
        
        # Szukaj numeru telefonu (przykładowe regex, można rozbudować)
        phone_tags = contact_soup.find_all('a', href = True)
        #print(phone_tags)

        pattern = r"\+?[0-9]{1,3}[-.\s]?[0-9]{2,3}[-.\s]?[0-9]{2,3}[-.\s]?[0-9]{2,4}[-.\s]?[0-9]{2,4}"

        for tag in phone_tags:
            #print(tag)
            tag = tag['href']
            if ("tel:" in tag or "+48" in tag): #and re.match(pattern, tag):  # Polski format numerów telefonu
                phone = tag
                print(phone)
                break
            elif len(re.findall(pattern, tag)) != 0:
                phone = re.findall(pattern, tag)[0]
                print(phone)
                break
        
        # Dodaj rekord do listy

        contacts.append({
            'Imię i Nazwisko': name,
            'E-mail': email,
            'Numer telefonu': phone,
            'Link do strony': link
        })

        print(contacts[-1])
    
    return contacts

# Funkcja do zapisania danych w CSV lub XLSX
def save_to_file(data, filename="logopeda_kontakty.csv"):
    df = pd.DataFrame(data)
    
    if filename.endswith(".csv"):
        df.to_csv(filename, index=False, sep = ';')
    elif filename.endswith(".xlsx"):
        df.to_excel(filename, index=False)
    else:
        print("Nieobsługiwany format pliku. Użyj .csv lub .xlsx")
        return

    print(f"Dane zostały zapisane w pliku: {filename}")


In [28]:
s = 'mailto:logopedawitkowska@gmail.com'
pattern = r'([a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.(com|org))'
email = re.findall(pattern, s)

#print(email)
#email = re.findall(r'\b[A-Za-z0-9._%+-]+@+[A-Za-z0-9._%+-]*\.+(com|org)$', s)
print(email[0])
                


('logopedawitkowska@gmail.com', 'com')


In [29]:
import re

# Regular expression to extract email after 'mailto:'
email_pattern = r'([a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.(com|org|pl))'

# Example usage:
test_string = "mailto:poczta@ratio.edu.pl"

# Using re.search to find and extract the email
match = re.search(email_pattern, test_string)
if match:
    print
    email = match.group(1)  # Extract the email from the first capture group
    print(f"Extracted email: {email}")
else:
    print("No valid email found.")

Extracted email: poczta@ratio.edu.pl


In [30]:
# Główna funkcja
def main():
    query = '"logopeda" "dzieci" "kontakt" "@gmail.com"'
    soup = google_search(query, num_results=200)  # Pobieranie 20 wyników wyszukiwania
    
    if soup:
        contacts = scrape_contacts(soup)
        
        if contacts:
            save_to_file(contacts, "logopeda_kontakty_with_tels.csv")
        else:
            print("Nie znaleziono danych kontaktowych")
    else:
        print("Błąd podczas scrapowania wyników wyszukiwania")

if __name__ == "__main__":
    main()

0
tel:601776153
{'Imię i Nazwisko': 'Logopeda dziecięcy i neurologopeda - Wrocław - dr I ...', 'E-mail': 'logopedawitkowska@gmail.com', 'Numer telefonu': 'tel:601776153', 'Link do strony': 'https://logopedawitkowska.pl/'}
1
tel:601776153
{'Imię i Nazwisko': 'Logopeda dziecięcy i neurologopeda - Wrocław - dr I ...', 'E-mail': 'logopedawitkowska@gmail.com', 'Numer telefonu': 'tel:601776153', 'Link do strony': 'https://logopedawitkowska.pl/'}
2
{'Imię i Nazwisko': 'Kontakt - Logopeda Kraków - mgr Katarzyna Jajuga', 'E-mail': None, 'Numer telefonu': None, 'Link do strony': 'https://www.pracownialogopedycznakrakow.pl/index.php/kontakt/'}
3
61551931047453
{'Imię i Nazwisko': 'Maria Petriczek gabinet neurologopedyczny', 'E-mail': None, 'Numer telefonu': '61551931047453', 'Link do strony': 'https://logopedanaruczaju.pl/'}
4
tel:+48572345861
{'Imię i Nazwisko': 'Logopedia Multisensoryczna | Magdalena Pasieka | Kraków ...', 'E-mail': 'logopedia.multisensoryczna@gmail.com', 'Numer telefonu': 'tel

In [31]:
import numpy as np
import math
csv = pd.read_csv('logopeda_kontakty_with_tels.csv', sep=';')
#print(math.isnan(csv.iloc[1][2]))
print(pd.value_counts(csv['E-mail'] == None))

E-mail
False    98
Name: count, dtype: int64


C:\Users\Jan\AppData\Local\Temp\ipykernel_20608\1715146517.py:5: FutureWarning: pandas.value_counts is deprecated and will be removed in a future version. Use pd.Series(obj).value_counts() instead.
  print(pd.value_counts(csv['E-mail'] == None))
